In [1]:
import pandas as pd
import numpy as np 
import altair as alt
import os
import pdfplumber
import requests
import glob
import re

### New York

In [2]:
pdf = pdfplumber.open("NYC-01-18-22.pdf")
page1 = pdf.pages[0].extract_text()
#string = pdf.pages[1].extract_tables()
page1 = page1.replace(u'\xa0', u'')
page1 = page1.split("\n")[6:]

page2 = pdf.pages[1].extract_text()
#string = pdf.pages[1].extract_tables()
page2 = page2.replace(u'\xa0', u'')
page2 = page2.split("\n")[6:]

nyc = page1 + page2

In [3]:
col_names = ["County","Number of Payments","Total Amount of Payments","Average Payment Amount"]
nyc_final_df = pd.DataFrame(columns=col_names)
for i in nyc:
    county=i.split(" ")[0]
    total_payments = (i.split()[-5]).strip()
    total_payments_amount = (i.split("$")[-2]).strip()
    total_payments_average_amount = (i.split("$")[-1]).strip()
    d = {"County": county, "Number of Payments": total_payments,
     "Total Amount of Payments": total_payments_amount,"Average Payment Amount": total_payments_average_amount}
    temp_df = pd.DataFrame(data=d, index=[0])
    nyc_final_df = nyc_final_df.append(temp_df, ignore_index=True)

In [5]:
nyc_final_df.at[30,"Number of Payments"] = 1050
nyc_final_df.at[29,"County"] = "New York"
nyc_final_df.at[40,"County"] = "Saint Lawrence"
nyc_final_df["State"] = "New York"
nyc_final_df = nyc_final_df[:-2]

In [14]:
for i in ['Number of Payments', 'Total Amount of Payments','Average Payment Amount']:
    nyc_final_df[i] = nyc_final_df[i].replace({',':""," ":""}, regex=True)
    nyc_final_df[i] = nyc_final_df[i].astype(float)

<ipython-input-14-e01f7f63f642>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_final_df[i] = nyc_final_df[i].replace({',':""," ":""}, regex=True)
<ipython-input-14-e01f7f63f642>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_final_df[i] = nyc_final_df[i].astype(float)


### missouri 

In [56]:
missouri = pd.read_csv("Missouri_ERAP_1_28_21.csv")
missouri = missouri.drop(columns=["Latitude (generated)","Longitude (generated)"])
missouri.rename(columns={"County ":"County","Number of Households":"Number of Payments","Award Amount":"Total Amount of Payments"},inplace=True)
missouri = missouri.replace(',','', regex=True)
missouri = missouri.replace('\$','', regex=True)
missouri["Total Amount of Payments"]= missouri["Total Amount of Payments"].str[:-3]
missouri["Number of Payments"] = missouri["Number of Payments"].astype(int)
missouri["Total Amount of Payments"] = missouri["Total Amount of Payments"].astype(int)
missouri["Average Payment Amount"] = round(missouri["Total Amount of Payments"] / missouri["Number of Payments"],2)
missouri["Average Payment Amount"] = round(missouri["Total Amount of Payments"] / missouri["Number of Payments"],2)
missouri["State"] = "Missouri"
missouri= missouri.fillna(0)

### wyoming 

In [58]:
wy = pd.read_csv("wyoming_ERAP.csv")
wy = wy.replace(',','', regex=True)
wy = wy.replace('\$','', regex=True)
wy["$ Paid"]= wy["$ Paid"].str[:-3]
wy.rename(columns={"# Of Apps Paid":"Number of Payments","$ Paid":"Total Amount of Payments",
                  "Applications submitted":"Total Applications Submitted"},inplace=True)
wy["State"] = "Wyoming"
wy["Average Payment Amount"] = round( wy["Total Amount of Payments"].astype(int)/wy["Number of Payments"],2)
wy.drop(columns=["Latitude (generated)","Longitude (generated)"],inplace=True)

## indiana 

In [15]:
pdf_indiana = pdfplumber.open("indiana_2.pdf")
page1_indy = pdf_indiana.pages[0].extract_text()
page1_indy = page1_indy.split("\n")[7:]
page2_indy = pdf_indiana.pages[1].extract_text()
page2_indy = page2_indy.split("\n")[:-4]
indy = page1_indy + page2_indy
indy = pd.DataFrame(indy)

In [13]:
indy["County"]=indy[0].str.split(" ").str[0]
indy["Number of Payments"] = indy[0].str.split(" ").str[1]
indy["Applications in Process"] = indy[0].str.split(" ").str[5]
indy["Total of Rental Assistance Issued"] = indy[0].str.split(" ").str[2]
indy["Total of Utility Assistance Issued"] = indy[0].str.split(" ").str[3]
indy["Total Assistance Paid and Obligated"] = indy[0].str.split(" ").str[4]
indy.drop(columns=[0],inplace=True,errors='ignore')
indy = indy.replace(',','', regex=True)
indy["Number of Payments"] = indy["Number of Payments"].astype(int)
indy["State"] = "Indiana"
indy["Total of Utility Assistance Issued"] = indy["Total of Utility Assistance Issued"].str[:-3]
indy["Total of Utility Assistance Issued"] = indy["Total of Utility Assistance Issued"].astype(int)
indy["Total of Rental Assistance Issued"] = indy["Total of Rental Assistance Issued"].str[:-3]
indy["Total of Rental Assistance Issued"] = indy["Total of Rental Assistance Issued"].astype(int)
indy["Total Amount of Payments"] = round(indy["Total of Utility Assistance Issued"] + indy["Total of Rental Assistance Issued"])
indy["Average Payment Amount"] = round(indy["Total Amount of Payments"] / indy["Number of Payments"],2)

## lousiana 

In [151]:
all_files = glob.glob(os.getcwd() + "/LA*")
column_names = ['County', 'Number of Payments', 'Total Amount of Payments', 'State',
       'Average Payment Amount']
LA_full = pd.DataFrame(columns = column_names)
for i in all_files:
    print(i)
    pdf_LA = pdfplumber.open(i)
    page1_LA = pdf_LA.pages[0].extract_text()
    page1_LA = page1_LA.split("\n")[7:]
    try: 
        page2_LA = pdf_LA.pages[1].extract_text()
        page2_LA = page2_LA.split("\n")[:-1]
    except:
        print("only one page")
        page2_LA = None
    if page2_LA:
        LA = page1_LA + page2_LA
        LA = pd.DataFrame(LA)
    else:
        LA = pd.DataFrame(page1_LA)
    LA[0] = LA[0].replace({'\$': '', ',': '','\.':""}, regex=True)
    LA["County"]=LA[0].str.split("Parish").str[0]
    LA["Number of Payments"] = LA[0].str.split("Parish").str[1]
    LA["Total Amount of Payments"] = LA["Number of Payments"].str.split(" ").str[2]
    LA.dropna(inplace=True)
    LA["Number of Payments"] = LA["Number of Payments"].str.split(" ").str[1]
    LA = LA.replace(',','', regex=True)
    LA["Number of Payments"] = LA["Number of Payments"].astype(int)
    LA["State"] = "Louisiana"
    LA["Average Payment Amount"] = round(LA["Total Amount of Payments"].astype(int) / LA["Number of Payments"].astype(int),2)
    LA.drop(columns=[0],inplace=True)
    LA_full = pd.concat([LA_full,LA])
for i in ['Number of Payments', 'Total Amount of Payments','Average Payment Amount']:
    LA_full[i] = LA_full[i].astype(int)
LA_full = LA_full.groupby("County").sum().reset_index()
LA_full["County"] = LA_full["County"] + "Parish"
LA_full["County"] = LA_full["County"].str.replace("St","St.")

/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_november.pdf
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_December.pdf
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_September.pdf
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_May.pdf
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_April.pdf
only one page
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_October.pdf
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_June.pdf
/Users/underriner/Desktop/work/clean_ERAP_work/ERAP_State_Extraction/LA_July.pdf


## New Hampshire

In [20]:
nh = pd.read_csv("New_Hampshire_ERAP.csv")
nh.rename(columns={"County1":"County","Average Amount Per Household":"Average Payment Amount",
         "Amount":"Total Amount of Payments"},inplace=True)
nh.drop(columns=["Latitude (generated)","Longitude (generated)"],inplace=True)
nh["State"] = "New Hampshire"
nh["Number of Payments"] = nh["Total Amount of Payments"]/nh["Average Payment Amount"]

## Kansas 

In [25]:
pdf_kansas = pdfplumber.open("kansas_erap.pdf")
page1_kansas = pdf_kansas.pages[0].extract_text()
page1_kansas = page1_kansas.split("\n")[7:]

page2_kansas = pdf_kansas.pages[1].extract_text()
page2_kansas = page2_kansas.split("\n")[2:]

page3_kansas = pdf_kansas.pages[2].extract_text()
page3_kansas= page3_kansas.split("\n")[2:]
kansas = page1_kansas + page2_kansas + page3_kansas

kansas.remove('3')
kansas.remove('1')
kansas.remove('2')

In [26]:
kansas_final = pd.DataFrame(kansas)
kansas_final["County"]=kansas_final[0].str.split(" ").str[0]
kansas_final["Applications in Process"] = kansas_final[0].str.split(" ").str[1]
kansas_final["Number of Payments"] = kansas_final[0].str.split(" ").str[2]
kansas_final["Total Amount of Payments"] = kansas_final[0].str.split(" ").str[3]
kansas_final.drop(columns=[0],inplace=True)
kansas_final = kansas_final.replace(',','', regex=True)
kansas_final = kansas_final.replace('\$','', regex=True)
kansas_final["Total Amount of Payments"] = kansas_final["Total Amount of Payments"].str[:-3]
kansas_final["Number of Payments"] = kansas_final["Number of Payments"].astype(int)
kansas_final["Total Amount of Payments"] = kansas_final["Total Amount of Payments"].astype(int)
kansas_final["Average Payment Amount"] = round(kansas_final["Total Amount of Payments"]/kansas_final["Number of Payments"],2)
kansas_final["State"] = "Kansas"

### Minnesota

In [29]:
minn = pd.read_csv("PUBLIC_RentHelpMN Tableau Data Doc (12.30.21 Data).csv")
minn = minn.rename(columns={"Apps By County":"Total Applications Received",
                    " totalRequest_sum ":"Total Funds Requested",
                    "Paid?":"Househols Served",
                    "Apps/Thousand Renters Under $35k":"apps_per_1k_renters_under_35k"})
minn["apps_under_50%_ami"] = minn["ami_status_L30%"] + minn["ami_status_L50%"]
minn = minn[["county_name","Total Applications Received","Total Funds Requested","Househols Served","apps_per_1k_renters_under_35k","apps_under_50%_ami"]]
minn["Total Funds Requested"]= minn["Total Funds Requested"].str.replace("$","")
minn["Total Funds Requested"] = minn["Total Funds Requested"].str.replace(",","")
minn["Total Funds Requested"] = minn["Total Funds Requested"].astype(float)
minn["State"] = "Minnesota"
minn.rename(columns={"county_name":"County",
                    "Househols Served":"Number of Payments"},inplace=True)

<ipython-input-29-108d0fe5734d>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  minn["Total Funds Requested"]= minn["Total Funds Requested"].str.replace("$","")


### Arizona

In [50]:
#from https://des.az.gov/erap-dashboard
arizona = pd.read_csv("Arizona_ERAP.csv")
arizona.rename(columns={"Approved Applications":"Number of Payments",
                    "Total Assistance":"Total Amount of Payments"},inplace=True)
arizona = arizona[["County","Number of Payments","Total Amount of Payments"]]
arizona["State"] = "Arizona"

for i in ['Number of Payments', 'Total Amount of Payments']:
    arizona[i] = arizona[i].replace({',':""," ":"","\$":""}, regex=True)
    arizona[i] = arizona[i].astype(float)
                                  
arizona["Average Payment Amount"] = round(arizona["Total Amount of Payments"]/arizona["Number of Payments"],2)
arizona = arizona[arizona.County != "Total"]

# California

In [21]:
cali = pd.read_csv("California_erap.csv")

In [22]:
cali["State"] = "California"
cali.rename(columns={"Counties":"County",
                    "Househols Served":"Number of Payments",
                    "Total Funds Paid":"Total Amount of Payments"},inplace=True)

cali["Average Payment Amount"] = cali["Total Amount of Payments"]/cali["Number of Payments"]

cali = cali[["County","State",
            "Number of Payments","Total Amount of Payments",
            "Average Payment Amount"]]

# Texas

In [23]:
texas = pd.read_csv("texas_ERAP.csv")

In [24]:
texas["State"] = "Texas"
texas.rename(columns={"Counties":"County",
                    "Households Assisted ":"Number of Payments",
                    "Assistance Received":"Total Amount of Payments"},inplace=True)
texas["Average Payment Amount"] = texas["Total Amount of Payments"]/texas["Number of Payments"]
texas = texas[["County","State",
            "Number of Payments","Total Amount of Payments",
            "Average Payment Amount"]]

# Nebraska 

In [64]:
nebraska = pd.read_csv("Nebraska_ERAP.csv")
nebraska.drop(columns=["County"],inplace=True)
nebraska["County"] = nebraska["Cnty Name"] + " County"
nebraska["State"] = "Nebraska"
nebraska.rename(columns={"Count of Application ID":"Number of Payments",
                    "Disbursed Amount":"Total Amount of Payments"},inplace=True)
nebraska["Average Payment Amount"] = round(nebraska["Total Amount of Payments"]/nebraska["Number of Payments"],2)
nebraska = nebraska[["County","State",
            "Number of Payments","Total Amount of Payments",
            "Average Payment Amount"]]

### Rhode Island 

In [67]:
ri

,Muni,Number of Payments,Total Amount of Payments,State,Average Payment Amount
0,Barrington,47,345005,Rhode Island,7340.531915
1,Bristol,98,644025,Rhode Island,6571.683673
2,Burrillville,109,684176,Rhode Island,6276.844037
3,Central Falls,609,3812859,Rhode Island,6260.852217
4,Charlestown,28,163628,Rhode Island,5843.857143
5,Coventry,168,1164730,Rhode Island,6932.916667
6,Cranston,874,6160398,Rhode Island,7048.510297
7,Cumberland,181,1056697,Rhode Island,5838.104972
8,East Greenwich,48,367142,Rhode Island,7648.791667
9,East Providence,472,3311400,Rhode Island,7015.677966


In [66]:
ri = pd.read_csv("RI_ERAP.csv",names=["Muni","Number of Payments","Total Amount of Payments"])
ri["Total Amount of Payments"]= ri["Total Amount of Payments"].str.replace("$","")
ri["Total Amount of Payments"] = ri["Total Amount of Payments"].str.replace(",","")
ri["Number of Payments"] = ri["Number of Payments"].str.replace(",","")
ri["Number of Payments"] = ri["Number of Payments"].astype(int)
ri["Total Amount of Payments"] = ri["Total Amount of Payments"].astype(int)
ri = ri.groupby("Muni").sum().reset_index()
ri["State"]= "Rhode Island"
ri["Average Payment Amount"] = ri["Total Amount of Payments"]/ri["Number of Payments"]
ri.Muni = ri.Muni.str.title()


<ipython-input-66-6c1564d93ed2>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ri["Total Amount of Payments"]= ri["Total Amount of Payments"].str.replace("$","")


In [29]:
ri["cousubnm14"] = ri["Muni"] + " town"

In [30]:
crosswalk = pd.read_csv("Rhode_island_County_muni_Crosswalk.csv")

In [31]:
crosswalk = crosswalk[["cntyname2","cousubnm14"]]

In [32]:
crosswalk.head(3)

,cntyname2,cousubnm14
0,2014 county name,County subdivision name 2014
1,Bristol RI,Barrington town
2,Bristol RI,Bristol town


In [33]:
ri = ri.merge(crosswalk,how="left")
#ri.to_csv("manually_fix_rhode_island.csv") # some counties were mapping with geocorr, googled and did manually 

In [34]:
ri = pd.read_csv("final_RI_ERAP.csv")

In [35]:
ri = ri[["State","cntyname2","Number of Payments","Total Amount of Payments","Average Payment Amount"]]
ri = ri.groupby("cntyname2").sum().reset_index()
ri["State"]= "Rhode Island"
ri["County"] = ri["cntyname2"].str[:-2]
ri.drop(columns=["cntyname2"],inplace=True)
ri = ri.round(2)
ri["County"] = ri["County"].str.strip()

In [36]:
### pull in states from Chae's manual scraping 

In [55]:
scraped = pd.read_csv("~/Desktop/work/clean_ERAP_work/raw_data/state_erap_manual_extraction_chae_1_26_22.csv")
#drop alaska census areas 
scraped = scraped[~scraped.County.str.contains("Census Area")]
scraped = scraped[scraped["Average Payment Amount"] != "#DIV/0!"]

In [56]:
for i in ["Average Payment Amount","Total Amount of Payments"]:
    scraped[i] = scraped[i].str.replace(',','', regex=True)
    scraped[i] = scraped[i].str.replace('$','', regex=True)
    

## combine 

In [57]:
final_erap = pd.concat([new_york_extract,missouri,wy,kansas_final,nh,louisiana,indy,minn,arizona,texas,cali,nebraska,ri,scraped])

In [58]:
final_erap_simple = final_erap[["County","State",
            "Number of Payments","Total Amount of Payments",
            "Average Payment Amount"]]

In [ ]:
#final_erap_full = final_erap[["County","State","Number of Payments","Total Amount of Payments",
#            "Number of Payments","Average Payment Amount",'Rent Arrears Application', 'Prospective Rent Applications',
#       'Utility Arrears Application','Applications in Process',
#       'Total of Rental Assistance Issued',
#       'Total of Utility Assistance Issued',
#       'Total Assistance Paid and Obligated']]

In [60]:
final_erap_simple.to_csv("~/Desktop/work/clean_ERAP_work/processed_data/1_26_22_ERAP_county_main.csv")
#final_erap_full.to_csv("11_12_21_ERAP_county_augmented.csv")

In [69]:
final_erap_simple["State"].nunique()

20

In [139]:
final_erap_simple

NameError: name 'final_erap_simple' is not defined

In [68]:
for i in sorted(final_erap_simple["State"].unique()):
    print(i, end=', ')

Alaska, Arizona, California, Colorado, Connecticut, Indiana, Kansas, Louisiana, Maine, Maryland, Michigan, Minnesota, Missouri, Nebraska, New Hampshire, New York, Rhode Island, Texas, Vermont, Wyoming, 